# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [190]:
# Load the libraries as required.
import pandas as pd
import numpy as np
import os
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_validate
from sklearn.linear_model import Ridge
print(os.getcwd())


c:\DataScienceInstitue\production\02_activities\assignments


In [191]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_df = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_df.info()
fires_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


# Get X and Y

Create the features data frame and target data.

In [192]:
X_features = fires_df.drop(['area'],axis=1)

print('Features',X_features.head())
X_features.columns

Features    coord_x  coord_y month  day  ffmc   dmc     dc  isi  temp  rh  wind  rain
0        7        5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0
1        7        4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0
2        7        4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0
3        8        6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2
4        8        6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0


Index(['coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi',
       'temp', 'rh', 'wind', 'rain'],
      dtype='object')

In [193]:
y_target = fires_df['area']
print('Target',y_target.tail())

Target 512     6.44
513    54.29
514    11.16
515     0.00
516     0.00
Name: area, dtype: float64


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [194]:
transformer = ColumnTransformer(
    transformers=[
        ('numeric_transfomer', StandardScaler(), ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi',
       'temp', 'rh', 'wind', 'rain'] ),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), ['month', 'day']),
    ],remainder='passthrough')
# ('imputer', SimpleImputer(missing_values=np.nan,strategy = 'median'),['coord_x', 'coord_y',  'ffmc', 'dmc', 'dc', 'isi',
#        'temp', 'rh', 'wind', 'rain'])
preproc1 = Pipeline(
    [
        ('preproc', transformer),
        
    ]
)
preproc1

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_transfomer',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])]))])

In [195]:
transformer = ColumnTransformer(
    transformers=[
        ('numeric_transfomer', StandardScaler(), ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi',
       'temp', 'rh', 'wind', 'rain'] ),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), ['month', 'day']),
    ],remainder='passthrough')
# ('imputer', SimpleImputer(missing_values=np.nan,strategy = 'median'),['coord_x', 'coord_y',  'ffmc', 'dmc', 'dc', 'isi',
#        'temp', 'rh', 'wind', 'rain'])
preproc2 = Pipeline(
    [
        ('preproc', transformer),
        ('transform', PowerTransformer(method='yeo-johnson'))
        
    ]
)
preproc2

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_transfomer',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('transform', PowerTransformer())])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [196]:
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
simpler_pipeline = Pipeline([
    ("PreProc1",preproc1),
    ("model",Ridge())
])
simpler_pipeline

Pipeline(steps=[('PreProc1',
                 Pipeline(steps=[('preproc',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('numeric_transfomer',
                                                                   StandardScaler(),
                                                                   ['coord_x',
                                                                    'coord_y',
                                                                    'ffmc',
                                                                    'dmc', 'dc',
                                                                    'isi',
                                                                    'temp',
                                                                    'rh',
                                                                    'wind',
                                                                    'rain']),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                   ['month',
                                                                    'day'])]))])),
                ('model', Ridge())])

In [197]:
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
# Pipeline B = preproc2 + baseline
advanced_pipeline = Pipeline([
    ("PreProc2",preproc2),
    ("model",linear_model.LinearRegression())
])
advanced_pipeline

Pipeline(steps=[('PreProc2',
                 Pipeline(steps=[('preproc',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('numeric_transfomer',
                                                                   StandardScaler(),
                                                                   ['coord_x',
                                                                    'coord_y',
                                                                    'ffmc',
                                                                    'dmc', 'dc',
                                                                    'isi',
                                                                    'temp',
                                                                    'rh',
                                                                    'wind',
                                                                    'rain']),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                   ['month',
                                                                    'day'])])),
                                 ('transform', PowerTransformer())])),
                ('model', LinearRegression())])

In [198]:
preprocessing = transformer

scoring = ['neg_root_mean_squared_error']
refit_scoring = ['neg_root_mean_squared_error']

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size = 0.2, random_state = 42)


In [199]:
simpler_pipeline.get_params()

{'memory': None,
 'steps': [('PreProc1', Pipeline(steps=[('preproc',
                    ColumnTransformer(remainder='passthrough',
                                      transformers=[('numeric_transfomer',
                                                     StandardScaler(),
                                                     ['coord_x', 'coord_y', 'ffmc',
                                                      'dmc', 'dc', 'isi', 'temp',
                                                      'rh', 'wind', 'rain']),
                                                    ('onehot',
                                                     OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                     ['month', 'day'])]))])),
  ('model', Ridge())],
 'verbose': False,
 'PreProc1': Pipeline(steps=[('preproc',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('numeric_transfomer',
              

In [200]:
# from sklearn.model_selection import GridSearchCV
# # parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
# param = {'clf_alpha':[0,0.5,1.0]}
# xyz = GridSearchCV(pipeline_A,param_grid=param,scoring=scoring,cv=5,refit='neg_root_mean_squared_error')
# xyz.fit(X_train,y_train)
# # res_simple = pd.DataFrame(res_simple_dict)
# # res_simple.head()


In [201]:
res_simple_dict = cross_validate(simpler_pipeline, X_features, y_target, cv=5, scoring = 'neg_root_mean_squared_error', return_train_score = True,error_score='raise')
res_simple = pd.DataFrame(res_simple_dict).assign(experiment = 1)
res_simple


,fit_time,score_time,test_score,train_score,experiment
0,0.070348,0.006106,-24.960102,-68.649246,1
1,0.009027,0.003964,-19.057200,-68.995511,1
2,0.007991,0.003993,-115.900972,-40.906086,1
3,0.009009,0.003007,-26.290585,-68.407243,1
4,0.008000,0.004001,-81.467494,-56.891656,1


In [202]:
res_advanced_dict = cross_validate(advanced_pipeline, X_features, y_target, cv=5, scoring = 'neg_root_mean_squared_error', return_train_score = True,error_score='raise')
advanced_result = pd.DataFrame(res_simple_dict).assign(experiment = 1)
advanced_result


,fit_time,score_time,test_score,train_score,experiment
0,0.070348,0.006106,-24.960102,-68.649246,1
1,0.009027,0.003964,-19.057200,-68.995511,1
2,0.007991,0.003993,-115.900972,-40.906086,1
3,0.009009,0.003007,-26.290585,-68.407243,1
4,0.008000,0.004001,-81.467494,-56.891656,1


In [203]:
# Pipeline C = preproc1 + advanced model


In [204]:
# Pipeline D = preproc2 + advanced model

    

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [205]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
param_grid = {'model__alpha': [0.1, 1.0, 10.0, 100.0]}
grid_cv = GridSearchCV(
    estimator=simpler_pipeline, 
    param_grid=param_grid, 
    scoring = scoring, 
    cv = 5,
    refit = "neg_root_mean_squared_error")
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('PreProc1',
                                        Pipeline(steps=[('preproc',
                                                         ColumnTransformer(remainder='passthrough',
                                                                           transformers=[('numeric_transfomer',
                                                                                          StandardScaler(),
                                                                                          ['coord_x',
                                                                                           'coord_y',
                                                                                           'ffmc',
                                                                                           'dmc',
                                                                                           'dc',
                                                                                           'isi',
                                                                                           'temp',
                                                                                           'rh',
                                                                                           'wind',
                                                                                           'rain']),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                                          ['month',
                                                                                           'day'])]))])),
                                       ('model', Ridge())]),
             param_grid={'model__alpha': [0.1, 1.0, 10.0, 100.0]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error'])

In [207]:
result = grid_cv.cv_results_
result_df = pd.DataFrame(result)
result_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error
0,0.009544,0.001978,0.003911,0.000188,0.1,{'model__alpha': 0.1},-40.788868,-19.521793,-33.413508,-85.262358,-26.827393,-41.162784,23.146488,4
1,0.008761,0.001549,0.003800,0.000376,1.0,{'model__alpha': 1.0},-40.844558,-19.267006,-33.560689,-85.244153,-26.534658,-41.090213,23.213966,3
2,0.008902,0.000872,0.005176,0.001341,10.0,{'model__alpha': 10.0},-40.731750,-18.649221,-33.074643,-85.110038,-25.805333,-40.674197,23.402919,2
3,0.008592,0.001348,0.004242,0.000847,100.0,{'model__alpha': 100.0},-40.123792,-17.348700,-30.367040,-84.815500,-24.471562,-39.425319,23.890587,1


In [208]:
grid_cv.best_params_

{'model__alpha': 100.0}

In [209]:
grid_cv.best_estimator_

Pipeline(steps=[('PreProc1',
                 Pipeline(steps=[('preproc',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('numeric_transfomer',
                                                                   StandardScaler(),
                                                                   ['coord_x',
                                                                    'coord_y',
                                                                    'ffmc',
                                                                    'dmc', 'dc',
                                                                    'isi',
                                                                    'temp',
                                                                    'rh',
                                                                    'wind',
                                                                    'rain']),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                   ['month',
                                                                    'day'])]))])),
                ('model', Ridge(alpha=100.0))])

# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

In [214]:
import pickle
with open('forest_fire_damage_area_prediction_model.pkl','wb') as file:
    pickle.dump(grid_cv,file)

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [215]:
import mlflow.sklearn
mlflow.set_tracking_uri('http://localhost:5001')
model_name = 'forest_fire_damage_area_prediction_model'
model_version = '6'
model_uri = f'models:/{model_name}/{model_version}'
pipe = mlflow.sklearn.load_model(model_uri)
pipe

RestException: RESOURCE_DOES_NOT_EXIST: Model Version (name=forest_fire_damage_area_prediction_model, version=6) not found

In [ ]:
import shap

data_transform = pipe.named_steps['preproc'].transform(X_test)

c:\Users\nages\miniconda3\envs\dsi_participant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.